
# 🌼 대규모 LLM을 활용한 지식 챗봇 개발 - 1차시(24.11.21)

---


In [167]:
import os
import shutil
import pandas as pd
import tensorflow as tf
import urllib3
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

In [168]:
lines = pd.read_csv('fra.txt', names=['src', 'tar', 'lic'], sep = '\t')
lines.head()

,src,tar,lic
0,Go.,Va !,CC-BY 2.0 (France) Attribution: tatoeba.org #2...
1,Go.,Marche.,CC-BY 2.0 (France) Attribution: tatoeba.org #2...
2,Go.,En route !,CC-BY 2.0 (France) Attribution: tatoeba.org #2...
3,Go.,Bouge !,CC-BY 2.0 (France) Attribution: tatoeba.org #2...
4,Hi.,Salut !,CC-BY 2.0 (France) Attribution: tatoeba.org #5...


In [169]:
del lines['lic']

In [170]:
print(f'전체 데이터 개수 : {len(lines)}')

전체 데이터 개수 : 232736


In [171]:
lines.head()

,src,tar
0,Go.,Va !
1,Go.,Marche.
2,Go.,En route !
3,Go.,Bouge !
4,Hi.,Salut !


In [172]:
lines = lines[:60000]
# 데이터 개수 줄이기
lines.sample(10)

,src,tar
24728,You better hurry.,Vous feriez mieux de vous dépêcher !
43219,Our team won 3 to 1.,Notre équipe a gagné trois à un.
9953,We're engaged.,Nous sommes fiancés.
34724,I'm feeling guilty.,Je me sens coupable.
12703,She is out now.,Elle est en déplacement en ce moment.
6695,Wait a while.,Attendez un moment.
17921,Tom is fearless.,Tom est intrépide.
29848,This is very good.,Très bien.
26051,Have you gone mad?,Êtes-vous devenue folle ?
28381,It's kind of hard.,C'est plutôt difficile.


In [173]:
lines.tar = lines.tar.apply(lambda x : '\t ' + x + ' \n')
# 시작을 의미하는 sos, 끝을 의미하는 eos

In [174]:
lines.sample(10)

,src,tar
133,No way!,\t Impossible ! \n
15606,I like the idea.,\t L'idée me plaît. \n
44634,Tom is having lunch.,\t Tom déjeune. \n
58437,I'll request a refund.,\t Je vais demander à être remboursé. \n
22052,It's a snowstorm.,\t C'est une tempête de neige. \n
57438,I hope no one sees us.,\t J'espère que personne ne nous voit. \n
12300,It's not there.,\t Il n'y est pas. \n
12566,Now I know why.,"\t Maintenant, je sais pourquoi. \n"
57436,I hope it'll be quiet.,\t J'espère que ce sera calme. \n
51776,Thanks for being you.,\t Merci d'être toi. \n


In [175]:
src_vocab = set()
for line in lines['src']:
    for char in line:
        src_vocab.add(char)
        # src 문자 집합 구축

In [176]:
print(src_vocab)

{'p', '"', 'L', 'X', 'd', ',', 'Q', '0', '9', 'e', '$', 'ï', 'Z', 'v', 'V', 't', '?', 'y', 'o', 'F', 'i', '3', '7', 'Y', 'w', 'G', 'P', '5', 'r', ':', '.', 'k', 'D', 'u', ' ', 'J', 'l', 'n', 'H', 'I', 'K', '%', 'A', 'b', '€', 'B', '1', '2', '’', 'M', 'm', 'c', "'", 'h', 's', 'N', 'U', '&', 'C', 'j', 'R', 'f', 'S', 'E', 'O', '-', 'z', '8', '4', 'g', 'a', '!', 'q', 'W', 'x', '/', 'T', 'é', '6'}


In [177]:
tar_vocab = set()
for line in lines['tar']:
    for char in line:
        tar_vocab.add(char)
        # tar 문자 집합 구축

In [178]:
print(tar_vocab)

{'p', '"', 'L', 'X', 'd', 'Q', ',', 'ù', '0', '9', 'e', '$', 'ç', 'ï', '\n', 'v', '\xa0', 'É', 'î', 'V', 't', 'y', '?', 'o', 'F', 'i', '3', '\u2009', '7', 'Y', 'w', 'G', 'P', '5', 'r', 'œ', ':', 'k', '.', 'D', 'u', ' ', 'À', 'J', 'l', 'n', '‽', 'I', 'H', 'K', '%', 'A', 'b', 'B', '1', '2', '’', 'M', 'm', '‘', 'c', '»', "'", 'h', 's', 'â', 'N', 'U', '&', 'Ê', 'C', 'j', 'R', 'Ô', 'f', 'Ç', 'S', '\t', 'E', 'O', '«', '-', 'z', '8', '4', 'g', 'a', '!', 'ê', 'q', 'à', '\u202f', 'W', 'x', 'û', 'ë', 'T', 'è', 'ô', 'é', '6'}


In [179]:
src_vocab_size = len(src_vocab) + 1
print(f'src 문장의 문자 집합 크기 : {src_vocab_size}')
# 인코더에 데이터를 넣을 때 사용될 차원의 크기

src 문장의 문자 집합 크기 : 80


In [180]:
tar_vocab_size = len(tar_vocab) + 1
print(f'tar 문장의 문자 집합 크기 : {tar_vocab_size}')
# 디코더에 데이터를 넣을 때 사용될 차원의 크기

tar 문장의 문자 집합 크기 : 102


In [181]:
src_to_index = {}

for i, word in enumerate(src_vocab):
    src_to_index[word] = i + 1
# 정수 인덱싱을 위한 인코딩 작업
# 패딩을 위한 +1

In [182]:
print(src_to_index)

{'p': 1, '"': 2, 'L': 3, 'X': 4, 'd': 5, ',': 6, 'Q': 7, '0': 8, '9': 9, 'e': 10, '$': 11, 'ï': 12, 'Z': 13, 'v': 14, 'V': 15, 't': 16, '?': 17, 'y': 18, 'o': 19, 'F': 20, 'i': 21, '3': 22, '7': 23, 'Y': 24, 'w': 25, 'G': 26, 'P': 27, '5': 28, 'r': 29, ':': 30, '.': 31, 'k': 32, 'D': 33, 'u': 34, ' ': 35, 'J': 36, 'l': 37, 'n': 38, 'H': 39, 'I': 40, 'K': 41, '%': 42, 'A': 43, 'b': 44, '€': 45, 'B': 46, '1': 47, '2': 48, '’': 49, 'M': 50, 'm': 51, 'c': 52, "'": 53, 'h': 54, 's': 55, 'N': 56, 'U': 57, '&': 58, 'C': 59, 'j': 60, 'R': 61, 'f': 62, 'S': 63, 'E': 64, 'O': 65, '-': 66, 'z': 67, '8': 68, '4': 69, 'g': 70, 'a': 71, '!': 72, 'q': 73, 'W': 74, 'x': 75, '/': 76, 'T': 77, 'é': 78, '6': 79}


In [183]:
tar_to_index = {}
for i, word in enumerate(tar_vocab):
    tar_to_index[word] = i + 1

In [184]:
print(tar_to_index)

{'p': 1, '"': 2, 'L': 3, 'X': 4, 'd': 5, 'Q': 6, ',': 7, 'ù': 8, '0': 9, '9': 10, 'e': 11, '$': 12, 'ç': 13, 'ï': 14, '\n': 15, 'v': 16, '\xa0': 17, 'É': 18, 'î': 19, 'V': 20, 't': 21, 'y': 22, '?': 23, 'o': 24, 'F': 25, 'i': 26, '3': 27, '\u2009': 28, '7': 29, 'Y': 30, 'w': 31, 'G': 32, 'P': 33, '5': 34, 'r': 35, 'œ': 36, ':': 37, 'k': 38, '.': 39, 'D': 40, 'u': 41, ' ': 42, 'À': 43, 'J': 44, 'l': 45, 'n': 46, '‽': 47, 'I': 48, 'H': 49, 'K': 50, '%': 51, 'A': 52, 'b': 53, 'B': 54, '1': 55, '2': 56, '’': 57, 'M': 58, 'm': 59, '‘': 60, 'c': 61, '»': 62, "'": 63, 'h': 64, 's': 65, 'â': 66, 'N': 67, 'U': 68, '&': 69, 'Ê': 70, 'C': 71, 'j': 72, 'R': 73, 'Ô': 74, 'f': 75, 'Ç': 76, 'S': 77, '\t': 78, 'E': 79, 'O': 80, '«': 81, '-': 82, 'z': 83, '8': 84, '4': 85, 'g': 86, 'a': 87, '!': 88, 'ê': 89, 'q': 90, 'à': 91, '\u202f': 92, 'W': 93, 'x': 94, 'û': 95, 'ë': 96, 'T': 97, 'è': 98, 'ô': 99, 'é': 100, '6': 101}


In [185]:
encoder_input = []
for line in lines['src']:
    encoded_line = []
    for char in line:
        encoded_line.append(src_to_index[char])
    encoder_input.append(encoded_line)
print(f'src문장의 정수 인코딩 {encoder_input[:5]}')

src문장의 정수 인코딩 [[26, 19, 31], [26, 19, 31], [26, 19, 31], [26, 19, 31], [39, 21, 31]]


In [186]:
decoder_input = []
for line in lines['tar']:
    decoded_line = []
    for char in line:
        decoded_line.append(tar_to_index[char])
    decoder_input.append(decoded_line)
print(f'tar문장의 정수 인코딩 {decoder_input[:5]}')

tar문장의 정수 인코딩 [[78, 42, 20, 87, 42, 88, 42, 15], [78, 42, 58, 87, 35, 61, 64, 11, 39, 42, 15], [78, 42, 79, 46, 42, 35, 24, 41, 21, 11, 42, 88, 42, 15], [78, 42, 54, 24, 41, 86, 11, 42, 88, 42, 15], [78, 42, 77, 87, 45, 41, 21, 42, 88, 42, 15]]


In [187]:
decoder_target = []
for line in lines['tar']:
    char_position = 0
    # 문자 위치를 추적할 수 있는 변수
    decoded_line = []
    for char in line:
        if char_position != 0:
            decoded_line.append(tar_to_index[char])
        char_position = char_position+1
    decoder_target.append(decoded_line)
print(f'디코더 target 문장의 정부 인코딩 {decoder_target[:5]}')

디코더 target 문장의 정부 인코딩 [[42, 20, 87, 42, 88, 42, 15], [42, 58, 87, 35, 61, 64, 11, 39, 42, 15], [42, 79, 46, 42, 35, 24, 41, 21, 11, 42, 88, 42, 15], [42, 54, 24, 41, 86, 11, 42, 88, 42, 15], [42, 77, 87, 45, 41, 21, 42, 88, 42, 15]]


In [188]:
lengths = []
for line in lines['src']:
    lengths.append(len(line))
max_src_len = max(lengths)
print(f'src 문장의 최대 길이 : {max_src_len}')

src 문장의 최대 길이 : 22


In [189]:
lengths = []
for line in lines['tar']:
    lengths.append(len(line))
max_tar_len = max(lengths)
print(f'tar 문장의 최대 길이 : {max_tar_len}')

tar 문장의 최대 길이 : 76


In [190]:
encoder_input = pad_sequences(encoder_input, maxlen=max_src_len, padding='post')
# 인코더에 들어갈 데이터 패딩 작업

In [191]:
decoder_input = pad_sequences(decoder_input, maxlen=max_tar_len, padding = 'post')
# 디코더에 들어갈 데이터 패딩 작업

In [192]:
decoder_target = pad_sequences(decoder_target, maxlen=max_tar_len, padding='post')
# 디코더의 정답 데이터로 사용될 데이터 패딩 작업

# 1. 인코더와 디코더의 문장 길이는 동일하게 맞추지 않아도 된다
# - 인코더 데이터는 인코더 데이터끼리, 디코더 데이터는 디코더 데이터끼리 맞추어 패딩하면 된다
# 2. 패딩을 앞에 둘까, 뒤에 둘까?
# - 문장 생성 시에는 문장 끝에 어떤 단어가 나올지가 더 중요하기 때문에 패딩을 앞에 두고
# - 문장의 순서, 시퀀스를 유지하는 더 중요한 경우에는 패딩을 뒤쪽에 둔다
# 3. 디코더의 target데이터에서는 <sos>가 제거되었는데 동일하게 패딩을 주어도 되는가?
# - 패딩은 부족한 부분을 0을 채워 동일한 길이로 맞추는 과정이기 때문에, target과 decoder의 길이 차이는 중요하지 않다

In [193]:
encoder_input = to_categorical(encoder_input)

In [194]:
decoder_input = to_categorical(decoder_input)

In [195]:
decoder_target = to_categorical(decoder_target)

In [196]:
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense
from tensorflow.keras.models import Model
import numpy as np

In [197]:
encoder_inputs = Input(shape = (None, src_vocab_size))
# (None, src_vocab_size)의 형태를 갖는 인풋을 주겠다
# None : 가변적인 입력 문장의 길이
# abc, de
# [[0,0,1],[0,1,0],[1,0,0]] # abc
# [[1,0,1], [1,1,1]] #de

In [198]:
encoder_lstm = LSTM(units=256, return_state=True)

In [199]:
encoder_outputs, state_h, state_c = encoder_lstm(encoder_inputs)
# LSTM은 은닉 상태와 셀 상태를 리턴한다

In [200]:
encoder_states = [state_h, state_c]
# context vector

In [201]:
decoder_inputs = Input(shape = (None, tar_vocab_size))
# 디코더는 인코더의 context vector를 초기 은닉 상태로 사용한다

In [202]:
decoder_lstm = LSTM(units=256, return_sequences=True, return_state=True)
# return_sequences : 모든 타임스텝의 출력
# return_state : 마지막 타임스텝의 은닉상태와 셀상태를 출력

In [203]:
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state = encoder_states)
# 디코더의 outputs 출력되는 구간

In [204]:
decoder_softmax_layer = Dense(tar_vocab_size, activation='softmax')
# 디코더의 출력은 타겟 단어 집합의 각 단어에 대한 확률 중 높은 값이어야한다

In [205]:
decoder_outputs = decoder_softmax_layer(decoder_outputs)

In [ ]:
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
# 모델에 입력할 데이터의 형태
model.compile(optimizer = 'rmsprop', loss='categorical_crossentropy')

In [207]:
model.fit(
    x = [encoder_input, decoder_input],
    # 모델에 입력할 데이터
    y = decoder_target,
    batch_size = 64,
    epochs = 10,
    validation_split = 0.2
)

Epoch 1/10
750/750 [==============================] - 248s 326ms/step - loss: 0.7183 - val_loss: 0.6322
Epoch 2/10
750/750 [==============================] - 268s 357ms/step - loss: 0.4389 - val_loss: 0.5080
Epoch 3/10
750/750 [==============================] - 252s 336ms/step - loss: 0.3679 - val_loss: 0.4485
Epoch 4/10
750/750 [==============================] - 212s 283ms/step - loss: 0.3272 - val_loss: 0.4142
Epoch 5/10
750/750 [==============================] - 218s 291ms/step - loss: 0.2998 - val_loss: 0.3894
Epoch 6/10
750/750 [==============================] - 262s 349ms/step - loss: 0.2795 - val_loss: 0.3739
Epoch 7/10
750/750 [==============================] - 277s 369ms/step - loss: 0.2637 - val_loss: 0.3600
Epoch 8/10
750/750 [==============================] - 272s 362ms/step - loss: 0.2509 - val_loss: 0.3546
Epoch 9/10
750/750 [==============================] - 276s 369ms/step - loss: 0.2401 - val_loss: 0.3471
Epoch 10/10
750/750 [==============================] - 253s 337m

In [208]:
# 번역 동작 step
# 1. 번역하려고 하는 input문장이 인코더에 들어가서 context벡터를 얻는다
# 2. context벡터와 <sos> (\t)를 디코더로 보낸다
# 3. 디코더가 <eos> (\n)이 나올 떄 까지 다음 문자를 예측 반복

encoder_model = Model(inputs = encoder_inputs, outputs = encoder_states)

In [ ]:
decoder_state_input_h = Input(shape = (256, )) # 은닉 상태
decoder_state_input_c = Input(shape = (256,)) # 셀상태

# 디코더 셀에서 각각 이전 시점의 상태를 저장하는 텐서
# 디코더의 은닉 상태와 셀 상태를 입력으로 받기 위한 텐서로, 디코더 LSTM의 hidden units크기와 동일하게 넣어주기

In [210]:
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

In [211]:
decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs, initial_state=decoder_states_inputs)
# 디코더 lstm모델이 입력값인 decoder_inputs와, 이전 상태인 decoder_States_inputs를 입력 받아
# 다음 단어를 예측하는 출력(decoder_outputs)와 새로운 은닉상태/셀 상태(state_h, state_c)를 계산

In [212]:
decoder_states = [state_h, state_c]

In [ ]:
decoder_outputs = decoder_softmax_layer(decoder_outputs)
# 디코더 출력값을 소프트맥스 레이어로 통과시켜 단어별 확률 분포 변환

In [ ]:
decoder_model = Model(inputs = [decoder_inputs] + decoder_states_inputs, outputs= [decoder_outputs] + decoder_states)
# 다음 단어의 확률 분포와 새로운 상태값을 반환

In [215]:
index_to_src = {}
for char, i in src_to_index.items():
    index_to_src[i] = char
    # key : 정수 인덱스, value가 문자로 되도록 뒤집어주기

print(index_to_src)

{1: 'p', 2: '"', 3: 'L', 4: 'X', 5: 'd', 6: ',', 7: 'Q', 8: '0', 9: '9', 10: 'e', 11: '$', 12: 'ï', 13: 'Z', 14: 'v', 15: 'V', 16: 't', 17: '?', 18: 'y', 19: 'o', 20: 'F', 21: 'i', 22: '3', 23: '7', 24: 'Y', 25: 'w', 26: 'G', 27: 'P', 28: '5', 29: 'r', 30: ':', 31: '.', 32: 'k', 33: 'D', 34: 'u', 35: ' ', 36: 'J', 37: 'l', 38: 'n', 39: 'H', 40: 'I', 41: 'K', 42: '%', 43: 'A', 44: 'b', 45: '€', 46: 'B', 47: '1', 48: '2', 49: '’', 50: 'M', 51: 'm', 52: 'c', 53: "'", 54: 'h', 55: 's', 56: 'N', 57: 'U', 58: '&', 59: 'C', 60: 'j', 61: 'R', 62: 'f', 63: 'S', 64: 'E', 65: 'O', 66: '-', 67: 'z', 68: '8', 69: '4', 70: 'g', 71: 'a', 72: '!', 73: 'q', 74: 'W', 75: 'x', 76: '/', 77: 'T', 78: 'é', 79: '6'}


In [216]:
index_to_tar = {}
for char, i in tar_to_index.items():
    index_to_tar[i] = char

print(index_to_tar)

{1: 'p', 2: '"', 3: 'L', 4: 'X', 5: 'd', 6: 'Q', 7: ',', 8: 'ù', 9: '0', 10: '9', 11: 'e', 12: '$', 13: 'ç', 14: 'ï', 15: '\n', 16: 'v', 17: '\xa0', 18: 'É', 19: 'î', 20: 'V', 21: 't', 22: 'y', 23: '?', 24: 'o', 25: 'F', 26: 'i', 27: '3', 28: '\u2009', 29: '7', 30: 'Y', 31: 'w', 32: 'G', 33: 'P', 34: '5', 35: 'r', 36: 'œ', 37: ':', 38: 'k', 39: '.', 40: 'D', 41: 'u', 42: ' ', 43: 'À', 44: 'J', 45: 'l', 46: 'n', 47: '‽', 48: 'I', 49: 'H', 50: 'K', 51: '%', 52: 'A', 53: 'b', 54: 'B', 55: '1', 56: '2', 57: '’', 58: 'M', 59: 'm', 60: '‘', 61: 'c', 62: '»', 63: "'", 64: 'h', 65: 's', 66: 'â', 67: 'N', 68: 'U', 69: '&', 70: 'Ê', 71: 'C', 72: 'j', 73: 'R', 74: 'Ô', 75: 'f', 76: 'Ç', 77: 'S', 78: '\t', 79: 'E', 80: 'O', 81: '«', 82: '-', 83: 'z', 84: '8', 85: '4', 86: 'g', 87: 'a', 88: '!', 89: 'ê', 90: 'q', 91: 'à', 92: '\u202f', 93: 'W', 94: 'x', 95: 'û', 96: 'ë', 97: 'T', 98: 'è', 99: 'ô', 100: 'é', 101: '6'}


In [217]:
def decode_sequence(input_seq):
    states_value = encoder_model.predict(input_seq)

    target_seq = np.zeros((1, 1, tar_vocab_size))
    # (1, 1, tar_vocab_size) 배열 생성
    target_seq[0, 0, tar_to_index['\t']] = 1
    # 0번째 문장의 0번째 단어의 \t에 해당하는 위치를 1로 설정

    stop_condition = False
    decoded_sentence = ''

    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)
        # target_seq : 현재 디코더의 입력 시퀀스
        # states_value : context vector
        # output_tokens : 다음 단어에 대한 예측 확률 분포, h : 은닉 상태, c : 셀 상태

        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        # 예측된 데이터 중에서 현재 타임 스텝의 마지막 값을 불러와서 그 중에 argmax(최대값)을 sample_token_index에 저장
        sampled_char = index_to_tar[sampled_token_index]

        decoded_sentence += sampled_char

        if (sampled_char == '\n' or len(decoded_sentence) > max_tar_len):
            stop_condition = True
            # eos에 도달하거나 최대 문장 길이를 넘어서면 반복을 중단

        target_seq = np.zeros((1, 1, tar_vocab_size))
        # 1, 1, tar_vocab_size의 배열 생성
        #(배치사이즈, 시퀀스 길이, 타겟 집합의 크기)
        target_seq[0, 0, sampled_token_index] = 1
        # 첫번째 배치, 첫번째 타임스텝에서 sampled_token_index에 해당하는 위치를 1로 설정

        states_value = [h, c]
        # 현재 시점의 상태를 다음 시점의 상태로 전달
        
    return decoded_sentence

In [221]:
for seq_index in [500, 1000]:
    input_seq = encoder_input[seq_index:seq_index +1]
    decoded_sentence = decode_sequence(input_seq)
    print(f'입력 문장 : {lines.src[seq_index]}')
    print(f'정답 문장 : {lines.tar[seq_index][2:len(lines.tar[seq_index])-1]}')
    print(f'번역 문장 : {decoded_sentence[1:len(decoded_sentence)]}')

1/1 [==============================] - 0s 21ms/step
입력 문장 : Be brief.
정답 문장 : Soyez brefs. 
번역 문장 : Soyez prudente ! 

1/1 [==============================] - 0s 29ms/step
입력 문장 : Forget it.
정답 문장 : Oubliez ça ! 
번역 문장 : Oubliez-le. 

